**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2023-2024/master/Assignment4/course_materials"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.0 MB/s eta 0:00:00


First let's read the RDF file

In [6]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS, FOAF
from rdflib.plugins.sparql import prepareQuery
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/rdf/example6.rdf", format="xml")
ns = Namespace("http://somewhere#")
VCARD = Namespace("http://www.w3.org./2001/vcard-rdf/3.0#")

**TASK 7.1: List all subclasses of "LivingThing" with RDFLib and SPARQL**

In [10]:
print("RDFLib:")
for s,p,o in g.triples((None, RDFS.subClassOf,ns.LivingThing)):
  print(s)
print("SPARQL:")

q1 = prepareQuery('''
    SELECT ?x WHERE {
      ?x rdfs:subClassOf ns:LivingThing.
    }
''', initNs = {"rdfs":RDFS, "ns" : ns}
)

for r in g.query(q1):
  print(r, r.x)

RDFLib:
http://somewhere#Person
http://somewhere#Animal
SPARQL
(rdflib.term.URIRef('http://somewhere#Person'),) http://somewhere#Person
(rdflib.term.URIRef('http://somewhere#Animal'),) http://somewhere#Animal


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [21]:
print("RDFLib:")

subclasses = []
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  subclasses.append(s)
subclasses.append(ns.Person)

for object in subclasses:
  for s,p,o in g.triples((None, RDF.type,object)):
    print(s)
print("SPARQL:")
q1 = prepareQuery('''
  SELECT ?x WHERE {
    {
        ?x a ns:Person.
    }
    UNION {
    ?class rdfs:subClassOf ns:Person.
    ?x a ?class.
    }
  }
  ''',
  initNs = {
      "rdfs": RDFS, "ns": ns}
)

for r in g.query(q1):
  print(r.x)

RDFLib:
http://somewhere#JaneSmith
http://somewhere#JimGonzalez
http://somewhere#SaraJones
http://somewhere#JohnSmith
SPARQL:
http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith
http://somewhere#JimGonzalez


**TASK 7.3: List all individuals of "Person" or "Animal" and all their properties including their class with RDFLib and SPARQL. You do not need to list the individuals of the subclasses of person**


In [25]:
print("RDFLib:")
subclasses = [ns.Person, ns.Animal]

for object in subclasses:
    for s,p,o in g.triples((None, RDF.type, object)):
      print("Individual: ", end="")
      print(s)
      print("Class type: ", end="")
      print(object)
      for s1,p1,o1 in g.triples((s, None, None)):
         print("- Property: ", end="")
         print(p1)

print()
print("SPARQL:")
q1 = prepareQuery('''
  SELECT ?individual ?property ?value WHERE {
    {
      ?individual a ns:Person.
    }
    UNION
    {
      ?individual a ns:Animal.
    }
    ?individual ?property ?value.
  }
  ''',
  initNs = { "ns": ns}
)

for r in g.query(q1):
  print(f"Individual: {r.individual} - Property: {r.property} - Value: {r.value}")

RDFLib:
Individual: http://somewhere#SaraJones
Class type: http://somewhere#Person
- Property: http://www.w3.org/1999/02/22-rdf-syntax-ns#type
- Property: http://www.w3.org/2001/vcard-rdf/3.0/Given
- Property: http://www.w3.org/2001/vcard-rdf/3.0/FN
- Property: http://www.w3.org/2001/vcard-rdf/3.0/Family
- Property: http://xmlns.com/foaf/0.1/knows
- Property: http://xmlns.com/foaf/0.1/knows
Individual: http://somewhere#JohnSmith
Class type: http://somewhere#Person
- Property: http://www.w3.org/1999/02/22-rdf-syntax-ns#type
- Property: http://www.w3.org/2001/vcard-rdf/3.0/Given
- Property: http://www.w3.org/2001/vcard-rdf/3.0/FN
- Property: http://www.w3.org/2001/vcard-rdf/3.0/Family
Individual: http://somewhere#RockySmith
Class type: http://somewhere#Animal
- Property: http://www.w3.org/1999/02/22-rdf-syntax-ns#type
- Property: http://www.w3.org/2001/vcard-rdf/3.0/Given
- Property: http://www.w3.org/2001/vcard-rdf/3.0/FN
- Property: http://www.w3.org/2001/vcard-rdf/3.0/Family
- Propert

**TASK 7.4:  List the name of the persons who know Rocky**

In [29]:
print("RDFLib:")

subclasses = [s for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person))]
subclasses.append(ns.Person)

for object in subclasses:
    for s,p,o in g.triples((None, RDF.type, object)):
       for s1,p1,o1 in g.triples((s, FOAF.knows, ns.RockySmith)):
          print(f"Individual: {s}")

print()

print("SPARQL:")

q1 = prepareQuery('''
  SELECT ?x WHERE {
    {
        ?x a ns:Person.
    }
    UNION {
    ?class rdfs:subClassOf ns:Person.
    ?x a ?class.
    }
    ?x foaf:knows ns:RockySmith.
  }
  ''',
  initNs = { "rdfs": RDFS, "ns": ns, "foaf": FOAF}
)

for r in g.query(q1):
  print(f"Individual: {r.x}")

RDFLib:
Individual: http://somewhere#JaneSmith
Individual: http://somewhere#SaraJones

SPARQL:
Individual: http://somewhere#SaraJones
Individual: http://somewhere#JaneSmith


**Task 7.5: List the entities who know at least two other entities in the graph**

In [31]:
print("RDFlib:")

entities = [s for s,p,o in g.triples((None, FOAF.knows, None))]
checked = []
for entity in entities:
   if entity not in checked and entities.count(entity) >= 2:
        print(f"Individual: {entity}")
        checked.append(entity)

print("SPARQL:")

q1 = prepareQuery('''
  SELECT ?x WHERE {
    {
        ?x foaf:knows ?o1.
        ?x foaf:knows ?o2.
        FILTER (?o1 != ?o2)
    }
  }
  GROUP BY ?x
  ''',
  initNs = { "foaf":FOAF }
)

for r in g.query(q1):
  print(f"Individual: {r.x}")

RDFlib:
Individual: http://somewhere#RockySmith
Individual: http://somewhere#SaraJones
Individual: http://somewhere#JaneSmith
SPARQL:
Individual: http://somewhere#RockySmith
Individual: http://somewhere#SaraJones
Individual: http://somewhere#JaneSmith
